![](../imagenes/header-genai.png)

# Lab 02 - Creando un Agente

Los LLMs son excelentes para responder preguntas generales. Sin embargo, por sí solos no bastan para ofrecer valor a tus clientes.

Para que sean capaces de responder a preguntas más complejas, se necesita información adicional y específica del usuario, como su ID de contrato, el último correo que envió a tu soporte o su informe de compras más reciente.

Los agentes están diseñados para superar este desafío. Son implementaciones de IA más avanzadas, compuestas por múltiples entidades (herramientas) especializadas en distintas acciones (por ejemplo, recuperar información o interactuar con sistemas externos).

En términos generales, tú construyes y expones un conjunto de funciones personalizadas para la IA. La LLM puede entonces razonar sobre qué información necesita reunir y qué herramientas utilizar para responder a las instrucciones recibidas.

<br><img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-functions-flow.png?raw=true" width="100%">

## Preparación

Para ejecutar los ejercicios, necesitamos conectar este notebook a un clúster/cómputo.

Simplemente siga los pasos a continuación:
1. En la esquina superior derecha, haga clic en **Connect**
2. Seleccione el clúster: **Serverless**

![](../imagenes/serverless.png)

# Configuración del ambiente

Vamos a comenzar seleccionando el catálogo y esquema donde se encuentran nuestros datos

In [0]:
%sql USE academia.ia

# Ejercicio 01 - Usando LLMs

## a. Usando Databricks Foundation Models

<img src="https://docs.databricks.com/en/_images/serving-endpoints-list.png" style="float: right; padding-left: 10px; padding-top: 15px" width=600>

Necesitamos un modelo capaz de interpretar el texto de las **opiniones** y extraer la información deseada. Para ello, vamos a utilizar **[Foundation Models](https://docs.databricks.com/en/machine-learning/foundation-models/index.html#pay-per-token-foundation-model-apis)**, que son grandes modelos de lenguaje (LLMs) ofrecidos por Databricks, y que pueden ser consultados bajo demanda, sin necesidad de implementación ni gestión de esos recursos.

Algunos modelos disponibles son:

- Meta Llama
- OpenAI GPT e GPT OSS
- Anthropic Claude Sonnet e Opus
- Google Gemini e Gemma
- GTE Large
- BGE Large

Ahora, vamos a verlos e funcionamiento!

1. En el **menú principal** a la izquierda, haz clic em **`Serving`**
2. En el modelo **Llama 4 Maverick**, haz clic en **`Use`**
3. Añada la siguiente instrucción:
    ```
    Clasifica el sentimiento de la siguiente opinión:
    Compré una tablet y estoy muy insatisfecho con la calidad de la batería. Dura muy poco tiempo y tarda mucho en cargarse.
    ```
    <br>
4. Haz clic en el icono **enviar**

Con esto, ¡ya podemos comenzar rápidamente a prototipar nuestros nuevos productos de datos!

**NOTA:** Aquí se accede a la lista completa de modelos: https://docs.databricks.com/aws/en/machine-learning/foundation-model-apis/supported-models

## b. Probando modelos en el AI Playground

<img src="https://docs.databricks.com/en/_images/ai-playground.gif" style="float: right; padding-left: 10px" width=600>

Para decidir cuál es el mejor modelo e instrucción para nuestro caso de uso, podemos utilizar el **[AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html)**.

De esta forma, podemos probar rápidamente diversas combinaciones de modelos e instrucciones a través de una interfaz intuitiva y elegir la mejor opción para utilizar en nuestro proyecto.

Vamos a realizar la siguiente prueba:

1. En el **menu principal** a la izquierda, haz clic en **`Playgroud`**
2. Haz clic en el **seletor de modelos** y selecciona el modelo **`Llama 4 Maverick`**
3. Haz clic en el icono **`Add endpoint`**
4. Haz clic en el **seletor de modelos** y selecciona el modelo **`GPT OSS 20B`**
5. Haz clic en el icono **`Add endpoint`**
6. Haz clic en el **seletor de modelos** y selecciona el modelo **`Gemma 3 12B`**
7. Añade la siguiente instrucción:
    ```
    Clasifique el sentimiento de la siguiente opinión:
    Compré una tablet y estoy muy insatisfecho con la calidad de la batería. Dura muy poco tiempo y tarda mucho en cargarse.
    ```
    <br>
8. Haz clic en el icono **enviar**

¡Ahora podemos comparar las respuestas, el tiempo y el costo de cada modelo para elegir el que mejor se adapte a las necesidades de nuestro proyecto!

# Ejercicio 02 - Definiendo las herramientas

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-functions-playground.gif?raw=true" style="float: right; padding-left: 10px" width=600>

Las herramientas de los agentes de IA permiten que los agentes realicen tareas más allá de la generación de lenguaje, como recuperar datos estructurados o no estructurados y ejecutar código personalizado.

Para crear una herramienta con **Mosaic AI Agent Framework**, puedes usar cualquier combinación de los siguientes métodos:

|Método| Descripción|
|---|---|
|**Funciones de Unity Catalog**| - Definidas y gestionadas en Unity Catalog con recursos de seguridad y cumplimiento integrados <br> - Crea un registro central de herramientas que pueden ser gobernadas como otros objetos de Unity Catalog <br> - Ofrecen mayor facilidad de descubrimiento y reutilización <br> - Ideales para aplicar transformaciones y agregaciones en grandes conjuntos de datos|
|**Herramientas de código del agente**| - Definidas directamente en el código del agente de IA <br> - Útiles para invocar APIs REST, ejecutar código arbitrario o utilizar herramientas de baja latencia <br> - No cuentan con gobernanza integrada ni con facilidad de descubrimiento de funciones|

## a. Ejecución de procesamientos arbitrarios

Las herramientas pueden ser muy útiles para definir las tareas que un agente puede ejecutar. Muchas veces, esas tareas son específicas de nuestro negocio y necesitamos definir cómo el agente las llevará a cabo. En esos casos, podemos utilizar una lógica arbitraria para desarrollar estas rutinas de forma más flexible.

Algunos casos de uso son:
* Cálculos matemáticos
* Tratamiento de texto
* Aplicación de reglas de negocio
* Validaciones

In [0]:
%sql CREATE OR REPLACE FUNCTION valida_id(
  cpf STRING COMMENT 'Número identificador'
)
RETURNS BIGINT
LANGUAGE PYTHON
COMMENT 'Utiliza esta función para validar el número identificador y convertirlo en número. Devuelve -1 si el identificador no es válido.'
AS
$$
  cpf = cpf.replace(".", "").replace("-", "")
  if len(cpf) != 11:
    return False
  elif not cpf.isdigit():
    return False
  else:
    d1 = (int(cpf[0])*1 + int(cpf[1])*2 + int(cpf[2])*3 + int(cpf[3])*4 + int(cpf[4])*5 + int(cpf[5])*6 + int(cpf[6])*7 + int(cpf[7])*8 + int(cpf[8])*9) % 11 % 10
    d2 = (int(cpf[0])*0 + int(cpf[1])*1 + int(cpf[2])*2 + int(cpf[3])*3 + int(cpf[4])*4 + int(cpf[5])*5 + int(cpf[6])*6 + int(cpf[7])*7 + int(cpf[8])*8 + d1*9) % 11 % 10
    if d1 == int(cpf[9]) and d2 == int(cpf[10]):
      return int(cpf)
    else:
      return -1
$$

In [0]:
%sql SELECT valida_id("111.111.111-11")

## b. Consultando datos estruturados

Para extraer el máximo valor de nuestros agentes, necesitamos que puedan acceder a nuestros datos corporativos. Solo con esta combinación podremos crear aplicaciones capaces de impactar al negocio.

### Acceso a datos del Lakehouse

Veamos cómo acceder a los datos de nuestras tablas Delta.

In [0]:
%sql CREATE OR REPLACE FUNCTION consultar_cliente(id BIGINT)
RETURNS TABLE (id_cliente BIGINT, nombre STRING, apellido STRING, num_pedidos INT)
COMMENT 'Use esta función para consultar los datos de un cliente'
RETURN SELECT id_cliente, nombre, apellido, num_pedidos FROM academia.ia.clientes c WHERE c.id_cliente = consultar_cliente.id

In [0]:
%sql SELECT * FROM consultar_cliente(11111111111)


<img src="https://docs.databricks.com/aws/en/assets/images/create-online-table-473e834357fdf2f576706b4a9100850f.png" style="float: right; width: 800px; margin-top: 70px; margin-left: 10px">

### Lakebase

Para escenarios en los que necesitamos menores latencias, también podemos utilizar **[Databricks Lakebase](https://docs.databricks.com/aws/en/oltp/)**, que consiste en una copia de solo lectura de una tabla Delta, que está almacenada en un formato orientado a filas, optimizado para el acceso **online**.

Lakebase es totalmente **serverless** y escala automaticamente la capacidad de procesamiento según la carga de solicitudes, proporcionando baja latencia y alto rendimiento en el acceso a los datos de cualquier escala.

Además, Lakebase ofrece **integración** con Mosaic AI Model Serving, Feature Serving y aplicaciones de generación aumentada por recuperación (RAG), donde se utiliza para realizar consultas rápidas de datos.


## c. Consultando datos no-estruturados

<img src="https://www.databricks.com/sites/default/files/2024-01/db-vector-search-image-01_0.png?v=1705100714" style="float: right; width: 800px; margin-left: 10px">

Sin embargo, muchas veces los datos a los que necesitamos acceder no son necesariamente estructurados o no buscamos una coincidencia exacta.

**[Databricks Vector Search](https://docs.databricks.com/aws/en/generative-ai/vector-search)** es una base de datos vectorial serverless, **integrada** de forma transparente en la Data Intelligence Platform.

A diferencia de otras bases de datos, Databricks Vector Search soporta la **sincronización automática** de los datos desde la fuente hacia el índice, eliminando el mantenimiento complejo y costoso de los pipelines.

Databricks Vector Search aprovecha las mismas herramientas de **seguridad y gobernanza** de datos que las organizaciones ya han implementado, ofreciendo mayor tranquilidad.

Gracias a su diseño serverless, Databricks Vector Search **escala** fácilmente para soportar miles de millones de embeddings y miles de consultas en tiempo real por segundo.


### i. Creando un Vector Search endpoint

Para crear el endpoint, siga los siguientes pasos:

1. En el **menú principal** a la izquierda, haz clic en **Compute**
1. En la parte superior, haz clic en la pestaña **Vector Search**
1. En la esquina superior derecha, haz clic en **Create endpoint**
1. Escriba el nombre: `academia-vs-endpoint`
1. Haz clic en **Confirm**

### ii. Consultando productos similares

Otro escenario interesante es el de las búsquedas por similitud.  

Por ejemplo, un usuario puede estar buscando alguna característica específica de un producto que no esté categorizada dentro de los filtros existentes de nuestro sitio web.

De esta manera, podemos buscar dentro de las descripciones de los productos y encontrar aquellos que sean más relevantes para el cliente, facilitando el descubrimiento de productos y aumentando las posibilidades de conversión.

Para crear el índice en Vector Search, siga los pasos a continuación:

1. En el **menu principal** de la izquierda, haz clic en **Catalog**
1. En la esquina superior izquierda, busque la tabla `productos`
1. En la esquina superior derecha, haz clic en  **Create** > **Vector search index**
1. Complete la siguiente información:
    - **Name:** productos_index
    - **Primary key:** id
    - **Embedding source column:** producto
    - **Embedding model:** databricks-gte-large-en
    - **Vector search endpoint:** academia-vs-endpoint
1. Haz clic en **Create**

In [0]:
%sql
CREATE OR REPLACE FUNCTION buscar_prod_sim(descripcion STRING)
RETURNS TABLE (id LONG, producto STRING, descripcion STRING, search_score DOUBLE)
COMMENT 'Esta función recibe la descripción de un producto, que es utilizada para buscar productos similares'
RETURN
SELECT id, producto, descripcion, search_score
FROM vector_search(
  index => 'academia.ia.productos_index',
  query_text => buscar_prod_sim.descripcion,
  query_type => 'HYBRID',
  num_results => 10
)
ORDER BY search_score DESC
LIMIT 3;

In [0]:
%sql SELECT * FROM buscar_prod_sim("El auricular DEF es un dispositivo de audio diseñado para ofrecer una experiencia de sonido inmersiva y de alta calidad. Con controladores de alta fidelidad y tecnología de cancelación de ruido, permite disfrutar de la música o los detalles de un pódcast sin distracciones. Además, su diseño ergonómico garantiza comodidad durante el uso prolongado.")

## d. Usando prompt engineering

Para personalizar el comportamiento de nuestros modelos de IA Generativa, podemos utilizar prompts curados por especialistas. De esta forma, conseguimos:

* Aprovechar mejor el conocimiento de expertos en IA o en un dominio de negocio específico para aumentar la **eficiencia** de los agentes 
* Promover la **reutilización** de estos activos entre proyectos
* **Democratizar** el acceso a la IA para usuarios menos avanzados

### Personalización de respuestas

Con todas las informaciones extraídas, podemos utilizarlas para generar sugerencias de respuestas personalizadas que aceleren el trabajo de nuestros equipos de atención.

Otro punto interesante es que, en este proceso, podemos aprovechar otras **informaciones estructuradas** que ya tengamos en nuestro entorno, como datos demográficos, psicográficos y el historial de compras, para personalizar aún más nuestras respuestas!

¡Vamos a ver cómo hacerlo!

In [0]:
%sql CREATE OR REPLACE FUNCTION generar_respuesta(nombre STRING, apellido STRING, num_pedidos INT, producto STRING, motivo STRING)
RETURNS TABLE(respuesta STRING)
COMMENT 'Si el cliente muestra insatisfacción con algún producto, use esta función para generar una respuesta personalizada'
RETURN SELECT AI_QUERY(
    'databricks-gpt-oss-120b',
    CONCAT(
        "Eres un asistente virtual de un e-commerce. Nuestro cliente, ", generar_respuesta.nombre, " ", generar_respuesta.apellido, " que compró ", generar_respuesta.num_pedidos, " productos este año estaba insatisfecho con el producto ", generar_respuesta.producto, 
        ", porque ", generar_respuesta.motivo, ". Proporcione un breve mensaje empático para el cliente, incluyendo la oferta de cambio del producto si está en conformidad con nuestra política de devoluciones. El cambio puede realizarse directamente por este asistente. ",
        "Quiero recuperar su confianza y evitar que deje de ser nuestro cliente.",
        "Escribe un mensaje con pocas frases.",
        "No agregues ningún texto además del mensaje.",
        "No añadas ninguna firma."
    )
)

In [0]:
%sql SELECT * FROM generar_respuesta("Juan", "Silva", 23, "tablet DEF", "duración de la bateria")

# Ejercicio 03 - Creando el agente

Para decidir cuál es el mejor modelo e instrucción para nuestro caso de uso, podemos utilizar el **[AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html)**.

De esta forma, podemos probar rápidamente diversas combinaciones de modelos e instrucciones a través de una interfaz intuitiva y elegir la mejor opción para utilizar en nuestro proyecto.

## a. Configurando el Agente

<img src="https://docs.databricks.com/en/_images/ai-playground.gif" style="float: right; margin-top:20px; margin-bottom:50px; padding-left: 10px" width=600>

Siga los pasos a continuación:

1. En el **menu principal** de la izquierda, haz clic en **Playground**
1. Haz clic en el **seletor de modelos** y seleccione el modelo **GPT OSS 120B**
1. Haz clic en el icono **Add endpoint**
1. Añada la siguiente instrucción en **Add system prompt**:<br>
    `Eres un asistente virtual de un e-commerce. Para responder a las preguntas, es necesario que el cliente proporcione un identificador válido. Si aún no tienes esa información, solicita el identificador educadamente. Tras validar el identificador, recuerda consultar los datos del cliente para personalizar sus respuestas. Si el identificador del cliente no existe en nuestra base, pide educadamente un nuevo identificador. Puedes responder preguntas sobre entrega, devolución de productos, estado de pedidos, entre otros. Si no sabes cómo responder la pregunta, di que no lo sabes. No inventes ni especules sobre nada. Siempre que se te pregunte sobre procedimientos, consulta nuestra base de conocimiento.`
    <br>
1. Haz clic en **Tools** > **Add tool**
1. Añada las rutas a sus **herramientas**: 
    - `academia.ia.valida_id`
    - `academia.ia.consultar_cliente`
    - `academia.ia.buscar_prod_sim`
    - `academia.ia.generar_respuesta`
1. Haz clic en el icono **Save** 

## b. Probando el Agente

Ahora podemos evaluar las respuestas, el tiempo y el costo de nuestro agente para entender si cumple con las necesidades de nuestro proyecto.

Envía los siguientes mensajes a tu agente:
1. Hola!
1. Mi identificador es 111.111.111-11
1. Compré una tablet DEF, pero la duración de la batería es muy corta
1. ¿Podrías sugerirme un producto mejor?

# Ejercicio 04 - Evaluando e Implementando Agentes

Después de crear el prototipo de nuestro agente, podemos avanzar a las etapas de evaluación e implementación.

## a. Evaluación del agente

Antes de poner cualquier agente en producción, igual que con cualquier otro tipo de software, es extremadamente importante evaluar su calidad. Para automatizar este proceso, podemos usar los **[built-in AI Judges](https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/concepts/judges/pre-built-judges-scorers)**. De esta forma, tenemos acceso a evaluadores desarrollados por el equipo de investigación de Databricks para tareas comunes de validación de calidad.

Siga los pasos a continuación para evaluar el agente:
1. En la parte superior central del Playground, haga clic en **Get code** > **Create agent notebook**
1. En el notebook creado, ejecute las celdas de las siguientes secciones para crear el agente y observar sus respuestas:
    - Prerequisites
    - Define the agent in code
    - Test the agent
1. En la sección **Log the agent as an MLflow model**, agregue el índice de Vector Search y la tabla de clientes a los recursos del agente:
```
from mlflow.models.resources import DatabricksTable, DatabricksVectorSearchIndex
resources.append(DatabricksTable('academia.ia.clientes'))
resources.append(DatabricksVectorSearchIndex('academia.ia.productos_index'))
```
4. Ejecute las celdas de la sección **Evaluate the agent with Agent Evaluation** para evaluar el rendimiento del agente utilizando los AI Judges.
1. Haz clic en **View evaluation results** para analizar el resultado de la evaluación.
1. Ejecute las celdas de la sección **Perform pre-deployment validation of the agent** para validar la ejecución del agente desde MLflow.

## b. Implementando el agente como una API REST

Ahora podemos llevar nuestro agente a producción. Para permitir una fácil integración con diversos sistemas y la ejecución del agente en tiempo real, lo implementaremos como una API REST.

Para facilitar este proceso, utilizaremos:
  - **Unity Catalog** para versionar y controlar el acceso a los artefactos del agente
  - **[Model Serving](https://docs.databricks.com/aws/en/machine-learning/model-serving/)** para crear un endpoint serverless para el agente

Siga los pasos a continuación para implementar el agente:
1. En la sección **Register the model to Unity Catalog**:
    - Complete las siguientes variables:
        - **catalog:** academia
        - **schema:** ia
        - **model_name:** agente_atencion_cliente
    - Ejecute las celdas para registrar el agente en Unity Catalog.
1. En la sección **Deploy the agent**, habilite la opción de scale to zero, como se muestra a continuación, y ejecute las celdas para implementar el agente como una API REST:
```
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, scale_to_zero=True, tags = {"endpointSource": "playground"})
```
3. Acceda al enlace generado junto a **View status** para visualizar el endpoint creado.
1. Después de que el estado del endpoint cambie a **Ready**, haz clic en el botón **Use** para hablar con su agente.

# ¡Felicidades!

¡Has completado el laboratorio de **Creando un agente**!

Ahora pasemos al siguiente laboratorio: [Lab 03 - Usando Batch Inference]($./Lab 03 - Usando Batch Inference)